In [18]:
import vtk
import numpy as np
import cantera as ct
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk

# load Tamaoki mechanism
for getting reaction equations and species names

In [ ]:
#create gas from original mechanism file gri30.cti
gas = ct.Solution('NH3_Tamaoki.yaml')
#reorder the gas to match pyJac
n2_ind = gas.species_index('N2')
specs = gas.species()[:]
gas = ct.Solution(thermo='IdealGas', kinetics='GasKinetics',
        species=specs[:n2_ind] + specs[n2_ind + 1:] + [specs[n2_ind]],
        reactions=gas.reactions())

# Read VTS file

In [20]:
path_read = "../xy_plane/pv_xy_z00001_0000900100.vts"
path_write = path_read.split('.vts')[0]+'_annotation.vts'

# VTSファイルを読み込む
reader = vtk.vtkXMLStructuredGridReader()
reader.SetFileName(path_read)  # 読み込むVTSファイルのパス
reader.Update()
structured_grid = reader.GetOutput()

In [21]:
list_labelEI = ['T']
list_labelEI.extend(gas.species_names)
list_labelEI

['T',
 'HE',
 'N2',
 'H2',
 'O2',
 'H',
 'O',
 'OH',
 'HO2',
 'H2O',
 'H2O2',
 'OH*',
 'N',
 'NH3',
 'NH2',
 'NH',
 'NNH',
 'NO',
 'N2O',
 'HNO',
 'HON',
 'H2NO',
 'HNOH',
 'NH2OH',
 'NO2',
 'HONO',
 'HNO2',
 'NO3',
 'HONO2',
 'N2H2',
 'H2NN',
 'N2H4',
 'N2H3',
 'AR']

In [22]:
def add_annotation(orig_label="index_maxEI", dest_label="Species_maxEI", list_annotaions=gas.species_names):

    # 既存のラベルデータ（argmax 結果）を取得
    argmax_array = structured_grid.GetPointData().GetArray(orig_label)  # 適宜変更
    if argmax_array is None:
        raise ValueError("Field 'argmax_result' not found in VTS file!")

    # NumPy 配列に変換
    argmax_np = vtk_to_numpy(argmax_array)

    # Annotation（カテゴリ名）に変換
    annotations = np.array([list_annotaions[int(val)-1] for val in argmax_np], dtype=object)
    print(np.unique(annotations))

    # VTK の文字列配列を作成
    vtk_annotations = vtk.vtkStringArray()
    vtk_annotations.SetName(dest_label)

    for label in annotations:
        vtk_annotations.InsertNextValue(label)

    # 既存の Structured Grid に Annotation を追加
    structured_grid.GetPointData().AddArray(vtk_annotations)

add_annotation(orig_label="index_maxEI", dest_label="Species_maxEI", list_annotaions=list_labelEI)
add_annotation(orig_label="index_maxPI", dest_label="Reactions_maxPI", list_annotaions=gas.reaction_equations())

['H2' 'H2NO' 'H2O' 'H2O2' 'HE' 'HNO' 'HO2' 'HON' 'HONO' 'N2' 'N2H2' 'N2H3'
 'N2H4' 'N2O' 'NH2' 'NH2OH' 'NH3' 'NO' 'NO2' 'O' 'O2' 'OH' 'T']
['2 NH2 (+M) <=> N2H4 (+M)' '2 NH2 <=> H2 + H2NN' '2 NH2 <=> NH + NH3'
 'H + H2O2 <=> H2O + OH' 'H + HNO <=> H2 + NO' 'H + NH2 <=> H2 + NH'
 'H + NH3 <=> H2 + NH2' 'H + O2 <=> O + OH' 'H2 + HE <=> 2 H + HE'
 'H2O2 (+M) <=> 2 OH (+M)' 'NH3 + M <=> H + NH2 + M' 'NNH <=> H + N2']


In [23]:
# VTSファイルとして保存
writer = vtk.vtkXMLStructuredGridWriter()
writer.SetFileName(path_write)  # 出力ファイル名
writer.SetInputData(structured_grid)
writer.Write()

print("Updated VTS file saved: " + path_write)

Updated VTS file saved: ../xy_plane/pv_xy_z00001_0000900100_annotation.vts
